# **Lab 5: Differential equations**
**Matteus Berg**

# **Abstract**


In this report, we implement an algorithm for solving the Initial Value Problem (IVP). Heun's method is chosen as the algorithm. The algorithm is then tested for both scalar IVP's and systems of IVP's. The algorithm is verified to converge.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license.

In [ ]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them.

In [ ]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *

#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

A scalar initial value problem is defined on page 280 in the book as

$\dot{u}(t) = f(u(t), t) \hspace{20pt} 0 < t \leq T$,

$u(0) = u_0$

Where $\dot{u}(t) = \frac{du}{dt}$. Obtaining exact solutions to these problems is a rarity. In general, solutions to this problem is approximated using numerical methods. One such numerical method is Heun's method, which is described on page 284 of the book. The time stepping for this method can be described with the two equations below:

$\tilde{u}_{n} = u_{n-1} + sf(u_{n-1}, t_{n-1})$

$u_n = u_{n-1} + \frac{s}{2}[f(t_{n-1}, u_{n-1}) + f(t_n, \tilde{u}_n)]$

Where $s = t_n - t_{n-1}$. $\tilde{u}_n$ is calculated using forward euler method. $\tilde{u}_n$ is then used to calculate $u_n$ with the trapetzoidal method. The trapetzoidal method is usually an implicit method, meaning that you need to solve an equation each time step when using it. However, by using euler's method to approximate $u_n$, Heun's method becomes an explicit method.

For a code implementation, it helps thinking of the algorithm performing two different steps:
1. Predictor step, doing the euler method
2. Corrector step, doing the trapetzoidal method

For system of initial value problems, we just increase the amount of equations. The method for solving them does not change with Heun's method.

# **Method**

In [ ]:
# step size
stepSize = 0.01

# forwardEuler method for one step
def forwardEuler(f, s, t, u0):
  return s*f(u0, t)

# Heun's method for one step
def heunMethod(f, s, t0, u0):
  t1 = t0+s
  # euler method. Predictor
  u1 = u0 + forwardEuler(f, s, t0, u0)

  # trapezoidal method. Corrector
  u = u0 + trapezoidal_rule(f, t0, t1, u0, u1)

  return u

# trapezoidal rule
def trapezoidal_rule(f, a, b, u0, u1):
  x0 = a
  x1 = b
  w0 = 1/2*(b-a)
  w1 = w0
  return f(u0, x0)*w0 + f(u1, x1)*w1

# initial value problem
def IVP(tn, f, u0):
  currentVal = u0
  for t in np.arange(0, tn, stepSize):
    currentVal = heunMethod(f, stepSize, t, currentVal)

  return currentVal

f = lambda u, t : t*t*t + 1
fprim = lambda u, t : 3*t*t
fSys = lambda u, t : np.array([-t, t*t])
fSysprim = lambda u, t : np.array([-1, 2*t])

finalTime = 5
initialValue = 1
print("t0 = 0, tN =", finalTime, "; step size =", stepSize)
print("Testing scalar initial value problem for u'(t) = 3*t^2 and u0 = 1.")
print("Expected value: 126")
print("Actual value: ",  IVP(finalTime, fprim, initialValue))
print(" ")
print("Testing system of initial value problems for u'(t) = [-1, 2*t] and u0 = [1, 1]")
print("Expected value: [-4, 26]")
print("Actual value: ",  IVP(finalTime, fSysprim, np.array([initialValue, initialValue])))



t0 = 0, tN =  5 ; step size =  0.01
Testing scalar initial value problem for u'(t) = 3*t^2 and u0 = 1.
Expected value: 126
Actual value:  126.00024999999752
 
Testing system of initial value problems for u'(t) = [-1, 2*t] and u0 = [1, 1]
Expected value: [-4, 26]
Actual value:  [-4. 26.]


# **Results**

Running the code in the method, we get that heun's method converges to the correct values when solving the initial value problems.

# **Discussion**

The implemented heun's method works as expected for both scalar IVP and systems of IVP.